# ✈️ FlightOnTime - Análisis Exploratorio de Datos (EDA)

## Hackathon de Aviación Civil - Predicción de Retrasos de Vuelos

---

**Objetivo:** Desarrollar un modelo de Machine Learning capaz de predecir si un vuelo despegará a tiempo o con retraso.

**Variable Objetivo:** `DEP_DEL15` → `is_delayed` (0 = Puntual, 1 = Retrasado ≥ 15 min)

**Features utilizadas:** 17 (Temporales, Operación, Distancia, Clima, Geográficas)

---

## 1. Configuración del Entorno

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12
sns.set_palette('husl')

# Configuración de Pandas
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

print('✅ Librerías cargadas correctamente')

## 2. Carga de Datos

In [ ]:
# Cargar dataset
DATA_PATH = Path('../0.0. DATASET ORIGINAL/dataset_prepared.parquet')

print(f'📁 Cargando datos desde: {DATA_PATH}')
df = pd.read_parquet(DATA_PATH)

# Muestreo para análisis exploratorio
SAMPLE_SIZE = 100_000
if len(df) > SAMPLE_SIZE:
    print(f'⚠️ Dataset muy grande ({len(df):,} registros). Tomando muestra de {SAMPLE_SIZE:,}...')
    df_sample = df.sample(n=SAMPLE_SIZE, random_state=42)
else:
    df_sample = df

print(f'\n📊 Dimensiones originales: {df.shape[0]:,} filas x {df.shape[1]} columnas')
print(f'📊 Muestra para EDA: {len(df_sample):,} registros')

In [ ]:
# Explorar primeras filas
print('\n📋 Primeras 5 filas del dataset:')
df_sample.head()

In [ ]:
# Información del dataset
print('📋 Información del Dataset:\n')
df_sample.info()

In [ ]:
# Estadísticas descriptivas
print('\n📊 Estadísticas descriptivas:')
df_sample.describe().round(2)

## 3. Variable Objetivo: DEP_DEL15 (Retrasos de Vuelos)

In [ ]:
# Análisis de la variable objetivo
target_col = 'DEP_DEL15'

print('📊 Distribución de la Variable Objetivo:')
print('=' * 50)
print(df_sample[target_col].value_counts())
print('\nPorcentajes:')
print(df_sample[target_col].value_counts(normalize=True).mul(100).round(2))

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de barras
colors = ['#2ecc71', '#e74c3c']
df_sample[target_col].value_counts().plot(kind='bar', ax=axes[0], color=colors)
axes[0].set_title('Distribución de Vuelos', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Tipo')
axes[0].set_ylabel('Cantidad')
axes[0].set_xticklabels(['Puntual (0)', 'Retrasado (1)'], rotation=0)

# Gráfico de pastel
df_sample[target_col].value_counts().plot(kind='pie', ax=axes[1], colors=colors, 
                                           autopct='%1.1f%%', startangle=90,
                                           labels=['Puntual', 'Retrasado'])
axes[1].set_title('Proporción de Vuelos', fontsize=14, fontweight='bold')
axes[1].set_ylabel('')

plt.tight_layout()
plt.savefig('../outputs/figures/eda_target_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n💡 Insight: Dataset desbalanceado - aprox. 81% puntuales vs 19% retrasados')

## 4. Features Temporales

In [ ]:
# Análisis temporal: retrasos por diferentes dimensiones
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Retrasos por mes (MONTH)
monthly_delay = df_sample.groupby('MONTH')[target_col].mean() * 100
monthly_delay.plot(kind='bar', ax=axes[0, 0], color='steelblue')
axes[0, 0].set_title('Tasa de Retrasos por Mes', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Mes')
axes[0, 0].set_ylabel('% Retrasos')
axes[0, 0].set_xticklabels(['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 
                            'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'], rotation=45)
axes[0, 0].axhline(y=df_sample[target_col].mean()*100, color='red', linestyle='--', label='Promedio')
axes[0, 0].legend()

# Retrasos por día de la semana (DAY_OF_WEEK)
weekly_delay = df_sample.groupby('DAY_OF_WEEK')[target_col].mean() * 100
weekly_delay.plot(kind='bar', ax=axes[0, 1], color='coral')
axes[0, 1].set_title('Tasa de Retrasos por Día de la Semana', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Día')
axes[0, 1].set_ylabel('% Retrasos')
axes[0, 1].set_xticklabels(['Lun', 'Mar', 'Mié', 'Jue', 'Vie', 'Sáb', 'Dom'], rotation=45)
axes[0, 1].axhline(y=df_sample[target_col].mean()*100, color='red', linestyle='--', label='Promedio')
axes[0, 1].legend()

# Retrasos por hora de salida (DEP_HOUR)
hourly_delay = df_sample.groupby('DEP_HOUR')[target_col].mean() * 100
hourly_delay.plot(kind='line', ax=axes[1, 0], marker='o', color='green', linewidth=2)
axes[1, 0].set_title('Tasa de Retrasos por Hora de Salida', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Hora')
axes[1, 0].set_ylabel('% Retrasos')
axes[1, 0].fill_between(hourly_delay.index, hourly_delay.values, alpha=0.3, color='green')
axes[1, 0].axhline(y=df_sample[target_col].mean()*100, color='red', linestyle='--', label='Promedio')
axes[1, 0].legend()

# Retrasos por año (YEAR)
yearly_delay = df_sample.groupby('YEAR')[target_col].mean() * 100
yearly_delay.plot(kind='bar', ax=axes[1, 1], color='purple')
axes[1, 1].set_title('Tasa de Retrasos por Año', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Año')
axes[1, 1].set_ylabel('% Retrasos')
axes[1, 1].axhline(y=df_sample[target_col].mean()*100, color='red', linestyle='--', label='Promedio')
axes[1, 1].legend()

plt.tight_layout()
plt.savefig('../outputs/figures/eda_temporal_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n💡 Insights Temporales:')
print(f'   - Hora con más retrasos: {hourly_delay.idxmax()}:00 ({hourly_delay.max():.1f}%)')
print(f'   - Hora con menos retrasos: {hourly_delay.idxmin()}:00 ({hourly_delay.min():.1f}%)')
print(f'   - Los vuelos de la tarde/noche tienen mayor tasa de retraso')

## 5. Variables de Operación (Aerolíneas y Aeropuertos)

In [ ]:
# Top 10 aerolíneas por tasa de retraso
carrier_stats = df_sample.groupby('OP_UNIQUE_CARRIER').agg({
    target_col: ['mean', 'count']
}).round(4)
carrier_stats.columns = ['tasa_retraso', 'total_vuelos']
carrier_stats['tasa_retraso'] = carrier_stats['tasa_retraso'] * 100
carrier_stats = carrier_stats.sort_values('tasa_retraso', ascending=False)

print('📊 Top 10 Aerolíneas por Tasa de Retraso:')
print(carrier_stats.head(10))

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Top 10 aerolíneas con más retrasos
top10 = carrier_stats.head(10)
top10['tasa_retraso'].plot(kind='barh', ax=axes[0], color='indianred')
axes[0].set_title('Top 10 Aerolíneas con Mayor Tasa de Retraso', fontsize=14, fontweight='bold')
axes[0].set_xlabel('% Retrasos')
axes[0].set_ylabel('Aerolínea')

# Volumen de vuelos por aerolínea
carrier_stats.sort_values('total_vuelos', ascending=False).head(10)['total_vuelos'].plot(
    kind='barh', ax=axes[1], color='steelblue'
)
axes[1].set_title('Top 10 Aerolíneas por Volumen de Vuelos', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Total Vuelos')
axes[1].set_ylabel('Aerolínea')

plt.tight_layout()
plt.savefig('../outputs/figures/eda_airlines_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Variables Climáticas (Gran Valor Agregado)

In [ ]:
# Análisis de variables climáticas vs retrasos
climate_cols = ['TEMP', 'WIND_SPD', 'PRECIP_1H', 'CLIMATE_SEVERITY_IDX']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for idx, col in enumerate(climate_cols):
    ax = axes[idx // 2, idx % 2]
    
    # Boxplot por tipo de vuelo
    df_sample.boxplot(column=col, by=target_col, ax=ax)
    ax.set_title(f'{col} por Tipo de Vuelo', fontsize=12, fontweight='bold')
    ax.set_xlabel('0=Puntual, 1=Retrasado')
    ax.set_ylabel(col)
    plt.suptitle('')

plt.tight_layout()
plt.savefig('../outputs/figures/eda_weather_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n💡 Insights Climáticos:')
for col in climate_cols:
    delayed_mean = df_sample[df_sample[target_col]==1][col].mean()
    ontime_mean = df_sample[df_sample[target_col]==0][col].mean()
    diff = delayed_mean - ontime_mean
    print(f'   - {col}: Retrasados {delayed_mean:.2f} vs Puntuales {ontime_mean:.2f} (Δ={diff:+.2f})')

## 7. Correlaciones

In [ ]:
# Matriz de correlación
numeric_cols = df_sample.select_dtypes(include=[np.number]).columns.tolist()
corr_matrix = df_sample[numeric_cols].corr()

plt.figure(figsize=(14, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='RdYlBu_r', center=0,
            fmt='.2f', square=True, linewidths=0.5)
plt.title('Matriz de Correlación de Variables Numéricas', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('../outputs/figures/eda_correlation_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

# Correlación con variable objetivo
print('\n📊 Correlación con Variable Objetivo (DEP_DEL15):')
target_corr = corr_matrix[target_col].drop(target_col).sort_values(key=abs, ascending=False)
print(target_corr.round(4))

## 8. Resumen de Features Seleccionadas

### Features del Modelo (17 total)

| Categoría | Features | Cantidad |
|-----------|----------|----------|
| Temporales | year, month, day_of_week, day_of_month, dep_hour, sched_minute_of_day | 6 |
| Operación | op_unique_carrier, origin, dest (encoded) | 3 |
| Distancia | distance | 1 |
| Clima | temp, wind_spd, precip_1h, climate_severity_idx, dist_met_km | 5 |
| Geográficas | latitude, longitude | 2 |

### Features Excluidas (Evitar Leakage)

- ❌ `DEP_DEL15` - Target
- ❌ `DEP_DELAY` - Contiene respuesta
- ❌ `STATION_KEY` - Llave técnica
- ❌ `FL_DATE` - Alta cardinalidad

In [ ]:
# Guardar resumen del EDA
summary = {
    'total_registros': int(len(df)),
    'total_columnas': int(len(df.columns)),
    'tasa_retraso_pct': float(df_sample[target_col].mean() * 100),
    'columnas': df.columns.tolist(),
    'features_modelo': [
        'year', 'month', 'day_of_week', 'day_of_month', 'dep_hour', 'sched_minute_of_day',
        'op_unique_carrier_encoded', 'origin_encoded', 'dest_encoded',
        'distance',
        'temp', 'wind_spd', 'precip_1h', 'climate_severity_idx', 'dist_met_km',
        'latitude', 'longitude'
    ],
    'hora_mas_retrasos': int(hourly_delay.idxmax()),
    'hora_menos_retrasos': int(hourly_delay.idxmin())
}

import json
with open('../outputs/metrics/eda_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print('✅ Resumen del EDA guardado en: outputs/metrics/eda_summary.json')
print(f'\n📊 Total de features del modelo: {len(summary["features_modelo"])}')